In [1]:
# !pip install --upgrade pip -q
# !pip install --upgrade sagemaker -q

In [7]:
import tarfile
import sagemaker
import tensorflow as tf
import tensorflow.keras as keras
import shutil
import os
import time
from tensorflow.keras.applications.resnet50 import ResNet50

role = sagemaker.get_execution_role()
sess = sagemaker.Session()
region = sess.boto_region_name
bucket = sess.default_bucket()
print('sagemaker version: '+sagemaker.__version__)
print('tensorflow version: '+tf.__version__)

sagemaker version: 2.15.1
tensorflow version: 2.1.0


In [11]:
def load_save_resnet50_model(model_path):
    model = ResNet50(weights='imagenet')
    shutil.rmtree(model_path, ignore_errors=True)
    model.save(model_path, include_optimizer=False, save_format='tf')

saved_model_dir = 'resnet50_saved_model' 
model_ver = '1'
model_path = os.path.join(saved_model_dir, model_ver)

load_save_resnet50_model(model_path)

INFO:tensorflow:Assets written to: resnet50_saved_model/1/assets


In [14]:
shutil.rmtree('model.tar.gz', ignore_errors=True)
!tar cvfz model.tar.gz -C resnet50_saved_model .

./
./1/
./1/variables/
./1/variables/variables.data-00000-of-00001
./1/variables/variables.index
./1/saved_model.pb
./1/assets/


In [16]:
from sagemaker.tensorflow.model import TensorFlowModel, TensorFlowPredictor

prefix = 'keras_models'
s3_model_path = sess.upload_data(path='model.tar.gz', key_prefix=prefix)
print(s3_model_path)

model = TensorFlowModel(model_data=s3_model_path, 
                        framework_version='1.15',
                        role=role,
                        predictor_cls = TensorFlowPredictor,
                        sagemaker_session=sess)

s3://sagemaker-us-west-2-453691756499/keras_models/model.tar.gz


### Deploy to GPU instance

In [17]:
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.g4dn.xlarge')

update_endpoint is a no-op in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


-------------!

In [18]:
## If you have an existing endpoint, create a predictor using the endpoint name

# from sagemaker.tensorflow.model import TensorFlowPredictor
# predictor = TensorFlowPredictor('ENDPOINT_NAME',
#                                sagemaker_session=sess)

In [19]:
def image_preprocess(img, reps=1):
    img = np.asarray(img.resize((224, 224)))
    img = np.stack([img]*reps)
    img = tf.keras.applications.resnet50.preprocess_input(img)
    return img

In [21]:
from PIL import Image 
import numpy as np
import json

img= Image.open('kitten.jpg')
img = image_preprocess(img, 5)
response = predictor.predict(data=img)
pred = np.array(response['predictions'][0]).argsort()[-5:][::-1] 

with open('imagenet_class_index.json', 'r') as f:
    labels = json.load(f)
    
for l in pred:
    print(labels[str(l)][1])

tiger_cat
tabby
Egyptian_cat
lynx
leopard
